In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple 
from itertools import count 
from PIL import Image 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

env = gym.make('CartPole-v1').unwrapped 
plt.ion()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
from matplotlib import font_manager
font_fname = 'C:/Windows/Fonts/malgun.ttf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()
plt.rcParams["font.family"] = font_family

In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward')) 
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:
class DQN(nn.Module):
    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        def conv2d_size_out(size, kernel_size=5, stride=2):
            return(size - (kernel_size-1)-1) // stride + 1

        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32 
        self.head = nn.Linear(linear_input_size, outputs)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

In [9]:
boarder = {'1': ' ' , '2': ' ' , '3': ' ' ,
           '4': ' ' , '5': ' ' , '6': ' ' ,
           '7': ' ' , '8': ' ' , '9': ' ' }
board_keys = []

for key in boarder:
    board_keys.append(key)

In [10]:
def visual_Board(board_num):
    print(board_num['1'] + '|' +     board_num['2'] + '|' + board_num['3'])
    print('-+-+-')
    print(board_num['4'] + '|' + board_num['5'] + '|' + board_num['6'])
    print('-+-+-')
    print(board_num['7'] + '|' + board_num['8'] + '|' + board_num['9'])

In [11]:
def game(): 
    turn = 'X'
    count = 0
    for i in range(8):
        visual_Board(boarder)
        print("당신 차례입니다," + turn + ". 어디로 이동할까요?")
        move = input()
        if boarder[move] == ' ':
            boarder[move] = turn
            count += 1
        else:
            print("이미 채워져 있습니다.\n어디로 이동할까요?")
            continue

        if count >= 5: 
            if boarder['1'] == boarder['2'] == boarder['3'] != ' ': 
                visual_Board(boarder)
                print("\n게임 종료.\n")
                print(" ---------- " + turn + "가 승리했습니다. -----------")
                break

            elif boarder['4'] == boarder['5'] == boarder['6'] != ' ': 
                visual_Board(boarder) 
                print("\n게임 종료.\n")
                print(" ---------- " + turn + "가 승리했습니다. -----------")
                break

            elif boarder['7'] == boarder['8'] == boarder['9'] != ' ':
                visual_Board(boarder)
                print("\n게임 종료.\n")
                print(" ---------- " + turn + "가 승리했습니다. -----------")
                break

            elif boarder['1'] == boarder['4'] == boarder['7'] != ' ': 
                visual_Board(boarder)
                print("\n게임 종료.\n")
                print(" ---------- " + turn + "가 승리했습니다. -----------")
                break

            elif boarder['2'] == boarder['5'] == boarder['8'] != ' ': 
                visual_Board(boarder)
                print("\n게임 종료.\n")
                print(" ---------- " + turn + "가 승리했습니다. -----------")
                break

            elif boarder['3'] == boarder['6'] == boarder['9'] != ' ': 
                visual_Board(boarder)
                print("\n게임 종료.\n")
                print(" ---------- " + turn + "가 승리했습니다. -----------")
                break

            elif boarder['1'] == boarder['5'] == boarder['9'] != ' ': 
                visual_Board(boarder)
                print("\n게임 종료.\n")
                print(" ---------- " + turn + "가 승리했습니다. -----------")
                break

            elif boarder['3'] == boarder['5'] == boarder['7'] != ' ': 
                visual_Board(boarder)
                print("\n게임 종료.\n")
                print(" ---------- " + turn + "가 승리했습니다. -----------")
                break

        if count == 9: 
            print("\n게임 종료.\n")
            print("동점입니다")

        if turn == 'X': 
            turn = 'Y'
        else:
            turn = 'X'

    if __name__ == "__main__":
        game()

In [12]:
python tic_tac_toe.py


SyntaxError: invalid syntax (2554316194.py, line 1)